In [1]:
# Импорт библиотеки для работы с clickhouse
import pandahouse as ph

In [2]:
# Парамаетры подключения к clickhouse
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

<div class = "alert alert-block alert-info"
<div>
Условие 1 задачи.

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной". Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин. 

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.NB! 

Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

In [5]:
# Решение 1 задачи
q_1 = '''
    SELECT
        uniqExact(st_id) AS count_good_fellows
    FROM
        (
            SELECT
                st_id,
                SUM(correct) AS count_correct_tasks
            FROM peas
            WHERE correct = 1
            GROUP BY
                st_id
            HAVING
                count_correct_tasks >= 20
        )
    '''
df_sql_1 = ph.read_clickhouse(query=q_1, connection=connection_default)
print(f'Кол-во усердных учеников равно {df_sql_1.count_good_fellows[0]}')

Кол-во усердных учеников равно 136


<div class = "alert alert-block alert-info"
<div>
Условие 2 задачи.

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты. 

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей: 

ARPU (Доход / Общее кол-во пользователей)

ARPAU (Доход / Кол-во активных пользоватлей)

CR в покупку (Пользователи, которые купили курс / Общее кол-во пользоватлей) 

СR активного пользователя в покупку (Пользователи, которые купили курс/ Общее кол-во пользвателей)

CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике (Пользователи, которые активные в математике и купили курс / Кол-во пользователей, которые купили кур по математике).

ARPU считается относительно всех пользователей, попавших в группы. Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах. Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

Активными считаю тех, кто решил >10 задач или активен по математике.

In [6]:
# Решение 2 задачи 
q = '''
SELECT 
    test_grp,
    SUM(money) / uniqExact(st_id) AS ARPU,
    SUM(money) / countIf(DISTINCT st_id, correct > 10 OR active_math = 1) AS ARPAU,
    SUM(purchase) / count(st_id) AS CR,
    SUM(purchase) / countIf(st_id, correct > 10 OR active_math = 1) AS CRA,
    sumIf(purchase, buy_math = 1 AND active_math = 1) / countIf(st_id, buy_math = 1) AS CR_math
FROM    
    (SELECT st_id, test_grp, subjects, correct, active_math, If(correct > 10, 1,0) AS active_user, money, If(money > 0, 1, 0) AS purchase, If(money > 0 AND subject = 'Math', 1, 0) AS buy_math
    FROM
        (SELECT *
        FROM    
            studs AS l
        LEFT JOIN
            (SELECT st_id, groupArray(subject) AS subjects, SUM(correct) AS correct, SUM(active_math) AS active_math
            FROM
                (SELECT
                    st_id,
                    subject,
                    SUM(correct) AS correct,
                    If(correct > 2 AND subject='Math', 1, 0) AS active_math
                FROM peas
                GROUP BY 
                    st_id,
                    subject)
            GROUP BY
                st_id) AS r
        ON l.st_id = r.st_id) AS l
    LEFT JOIN 
        final_project_check AS r
    ON l.st_id = r.st_id)
GROUP BY
    test_grp
    '''
df_sql = ph.read_clickhouse(query=q, connection=connection_default)
print(df_sql)

  test_grp          ARPU         ARPAU        CR       CRA  CR_math
0  control   4540.983607  10335.820896  0.058442  0.131387    0.375
1    pilot  11508.474576  31146.788991  0.140523  0.361345    0.250
